In [ ]:
%matplotlib inline


Default Mode Network extraction of AHDH dataset
===============================================

Full step-by-step example of fitting a GLM to a seed extract on the
Posterior Cingulate Cortex data and saving the results.

More specifically:

1. A sequence of fMRI volumes are loaded
2. A design matrix with the Posterior Cingulate Cortex seed is defined
4. A GLM is applied to the dataset (effect/covariance,
   then contrast estimation)
5. The Default Mode Network is displayed




In [ ]:
import numpy as np

from nilearn import datasets, plotting
from nilearn.input_data import NiftiSpheresMasker

from nistats.first_level_model import FirstLevelModel
from nistats.design_matrix import make_first_level_design_matrix

Prepare data and analysis parameters
-------------------------------------
Prepare data



In [ ]:
adhd_dataset = datasets.fetch_adhd(n_subjects=1)

# Prepare timing
t_r = 2.
slice_time_ref = 0.
n_scans = 176

# Prepare seed
pcc_coords = (0, -53, 26)

Estimate contrasts
------------------
Specify the contrasts



In [ ]:
seed_masker = NiftiSpheresMasker([pcc_coords], radius=10, detrend=True,
                                 standardize=True, low_pass=0.1,
                                 high_pass=0.01, t_r=2.,
                                 memory='nilearn_cache',
                                 memory_level=1, verbose=0)
seed_time_series = seed_masker.fit_transform(adhd_dataset.func[0])
frametimes = np.linspace(0, (n_scans - 1) * t_r, n_scans)
design_matrix = make_first_level_design_matrix(frametimes, hrf_model='spm',
                                               add_regs=seed_time_series,
                                               add_reg_names=["pcc_seed"])
dmn_contrast = np.array([1] + [0]*(design_matrix.shape[1]-1))
contrasts = {'seed_based_glm': dmn_contrast}

Perform first level analysis
----------------------------
Setup and fit GLM



In [ ]:
first_level_model = FirstLevelModel(t_r=t_r, slice_time_ref=slice_time_ref)
first_level_model = first_level_model.fit(run_imgs=adhd_dataset.func[0],
                                          design_matrices=design_matrix)

contrast estimation



In [ ]:
print('Contrast seed_based_glm computed.')
z_map = first_level_model.compute_contrast(contrasts['seed_based_glm'],
                                           output_type='z_score')

# Saving snapshots of the contrasts
filename = 'dmn_z_map.png'
display = plotting.plot_stat_map(z_map, threshold=3.0, title='Seed based GLM',
                                 cut_coords=pcc_coords)
display.add_markers(marker_coords=[pcc_coords], marker_color='g',marker_size=300)
display.savefig(filename)
print("Save z-map in '{0}'.".format(filename))